In [1]:
import os
import sys
import utils
import random
import dataset
import train
import network
import binvox_rw
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf

In [2]:
# load data
shapenet=dataset.ShapeNet()
# shapenet.batch_size=3

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
x=tf.placeholder(tf.float32,[shapenet.batch_size,137,137,3])
feature_vector=network.encoder(x)
hidden_state=network.recurrent_module(feature_vector)
prediction=network.decoder(hidden_state)
y=tf.placeholder(tf.float32,[shapenet.batch_size,32,32,32])
optimizer = train.evaluate(prediction,y)

In [ ]:
saver = tf.train.Saver()

In [ ]:
# start new or continue exisiting training session
trainer=train.Trainer()
trainer.

# train network
epoch=5
sess=tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
loss=[]       
for e in range(1,epoch+1): 
    common_dir="./{}/epoch_{:03d}/batch_{:03d}".format(train_dir,e,0)
    if not os.path.isdir(common_dir):
        os.makedirs(common_dir)
        save_path = saver.save(sess,"{}/model.ckpt".format(common_dir))
        print("{}: inital config for epoch saved at epoch_{:03d}/batch_{:03d}".format(train_dir,e,0))
    else:
        print("possibliy corrupt intial param config manaully create inital condition")
    batch_number=1
    train=shapenet.next_train_batch()
    while(train is not None):
        common_dir="./{}/epoch_{:03d}/batch_{:03d}".format(train_dir,e,batch_number)
        if not os.path.isdir(common_dir):
            train_X=dataset.load_dataset(train[:,0])
            train_Y=dataset.load_labels(train[:,1])
            fd={x:train_X , y: train_Y};
            l=sess.run([loss_voxel],feed_dict=fd)
            loss.append(l) 
            os.makedirs(common_dir)
            # record parameters and generate plots
            
            print("{}: epoch {:03d} batch {:03d} ".format(train_dir,e,batch_number))
        else:
            prev_dir="./{}/epoch_{:03d}/batch_{:03d}".format(train_dir,e,batch_number-1)
            saver.restore(sess,"{}/model.ckpt".format(prev_dir))
            continue
        
        # update
        train=shapenet.next_train_batch()
        batch_number+=1

In [ ]:
self.writer = tf.summary.FileWriter("./logs/")
self.writer.add_graph(sess.graph)  

In [ ]:
# model output as a voxelized image
out=tf.cast(tf.argmax(y_hat,axis=4),dtype=tf.float32)
out=sess.run(out,feed_dict=feed_dict)
outvoxel=binvox_rw.Voxels(out,out.shape,[0,0,0],1,'xzy')
with open("out/{i}.binvox",'w') as f:
    outvoxel.write(f)